# VacationPy
----

In [65]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from IPython.display import display

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [66]:
part1_df = pd.read_csv("../output_data/city_data.csv")
part1_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Clyde River,70.4692,-68.5914,-5.57,84,100,2.30,CA,1650126710
1,Tabuk,17.4189,121.4443,73.44,89,100,4.32,PH,1650126711
2,Punta Arenas,-53.1500,-70.9167,46.51,61,40,3.44,CL,1650126711
3,Busselton,-33.6500,115.3333,59.90,75,75,11.70,AU,1650126712
4,Port-Gentil,-0.7193,8.7815,82.27,76,91,7.16,GA,1650126712


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [67]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [89]:
# Bring in location data pairs, humidity data, define map and layer, and add layer to map
locations = part1_df[["Lat","Lng"]]
humid = part1_df["Humidity"]

fig = gmaps.figure(center=(0,0),zoom_level=1.5)

humidLayer = gmaps.heatmap_layer(locations, weights=humid,
                                dissipating = False, max_intensity = humid.max(),
                                point_radius = 2)
fig.add_layer(humidLayer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [90]:
part1_df = part1_df.dropna(how='all')

In [91]:
ideal_cond = part1_df.loc[(part1_df["Max Temp"] >= 70) & (part1_df["Max Temp"] < 80) &
                          (part1_df["Wind Speed"] < 10) & (part1_df["Cloudiness"] == 0),:]

### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [92]:
# Create dataframe and empty column
hotel_df = pd.DataFrame(ideal_cond[["City","Country","Lat","Lng"]])
hotel_df["Hotel Name"] = ""

In [93]:
# Loop through dataframe and add nearby hotel result

# Base URL with parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key
}

#----BEGIN Loop----#
for index, row in hotel_df.iterrows():
    
    # Variable set up
    lati = row["Lat"]
    long = row["Lng"]
    city = row["City"]
    params["location"] = f"{lati},{long}"
    
    # API request
    response = requests.get(base_url, params=params).json()
    
    results = response["results"]
    
    # Try/Except to add hotel names and skip if error
    try:
        hotel_df.loc[index,"Hotel Name"] = results[0]["name"]
        
    except (KeyError, IndexError):
        print(f"Missing field/result for {city}.")
#----END Loop-----#

In [94]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [78]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))